# Water Consumtion in Sorono
## Stage 3

In this stage I have divided the data table to one main and three child tables that will be joined through primary-foriegn keys.
- The main table, I removed all text from it and placed foriegn keys instead
- The other three tables I have hot-encoded them, each with its suitable encoding
   - The `Usage` table was encoded based on the properties of its values.
   - The `User` table was encoded based on the properties of its values.
   - The `Gauge` table was encoded based on teh one-hot-encoding method.


In [46]:
# Import the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import time
from datetime import timedelta
%matplotlib inline


#### Reading the tables

In [2]:
# Reading the tables
main_df = pd.read_csv('Aqua_ref.CSV.gz', compression='gzip')
main_df.sample(3)

,Condition,Case,Usage_ref,User_ref,Gauge_ref,Diameter,Year,Month,Reference,Consumption
9096971,211,718,3,6,1,0.5,14,6,814,19
4682989,441,10279,5,4,1,0.5,11,10,1415,25
10100951,441,8757,5,4,1,0.5,15,1,1029,16


In [3]:
Usage_df = pd.read_csv('Usage_encoded.CSV')
User_df = pd.read_csv('User_encoded.CSV')
Gauge_df = pd.read_csv('Gauge_encoded.CSV')
display(Usage_df.sample(3), User_df.sample(3), Gauge_df.sample(3))

,Usage_id,Usage_text,industry,housing,agriculture,commerce,reserve,government,low_density,mid_density,high_density
0,0,Downtown,0,1,0,1,0,0,0,0,0
1,1,Government Reserve,0,0,0,0,1,1,0,0,0
8,8,Medium Risk Industry,1,0,0,0,0,0,0,1,0


,User_id,User_text,u_industry,u_domestic,u_social,u_commerce,u_low_density,u_mid_density,u_high_density,u_residential
4,4,Medium Domestic,0,1,0,0,0,1,0,0
2,2,Industrial,1,0,0,0,0,0,0,0
5,5,Residential Domestic,0,1,0,0,0,0,0,1


,Gauge_id,Gauge_text,Gauge_names,Gauge_Abb,Gauge_Actaris,Gauge_Adcom,Gauge_Alfa,Gauge_Aquarius,Gauge_Arad,Gauge_Av3Stars,...,Gauge_Otras_Desconocida,Gauge_Precisa_Ii,Gauge_Recordal,Gauge_Sappel,Gauge_Sappel_Aquarius,Gauge_Sensus,Gauge_Shlumberger,Gauge_Tonhy,Gauge_Yt_Md15,Gauge_Zenner_Half_In
36,36,PRECISA II,Precisa_Ii,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34,34,Kent ABB,Kent_Abb,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,8,Cicasa NG 1/2,Cicasa_Ng_Half_In,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Merging the tables

In [4]:
df = main_df.merge(Usage_df, 
      left_on='Usage_ref', right_on='Usage_id', 
      how='outer').drop(columns=['Usage_ref', 
                                 'Usage_id', 'Usage_text']).merge(User_df, 
      left_on='User_ref', right_on='User_id', 
      how='outer').drop(columns=['User_ref', 
                                 'User_id', 'User_text']).merge(Gauge_df, 
      left_on='Gauge_ref', right_on='Gauge_id', 
      how='outer').drop(columns=['Gauge_ref',
                                 'Gauge_id', 'Gauge_text', 'Gauge_names'])#.rename( \
      #columns={'Usage_text':'Usage', 'User_text':'User', 'Gauge_text':'Gauge',})

In [5]:
# to freeup the memory
main_df = None

In [6]:
df.sample(10)

,Condition,Case,Diameter,Year,Month,Reference,Consumption,industry,housing,agriculture,...,Gauge_Otras_Desconocida,Gauge_Precisa_Ii,Gauge_Recordal,Gauge_Sappel,Gauge_Sappel_Aquarius,Gauge_Sensus,Gauge_Shlumberger,Gauge_Tonhy,Gauge_Yt_Md15,Gauge_Zenner_Half_In
1324365,149,13889,0.5,11,8,900,20,0,1,0,...,0,0,0,0,0,0,0,0,0,0
8512045,164,1984,0.5,11,10,0,6,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6131194,196,305,0.5,15,10,31,10,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2954192,441,5769,0.5,13,1,655,10,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5923813,774,300,0.5,13,11,159,24,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2238618,281,583,0.5,9,9,392,15,0,1,0,...,0,0,0,0,0,0,0,0,0,0
11260263,144,791,0.5,14,5,1805,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2175574,149,6548,0.5,15,10,420,19,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9717669,197,935,0.5,12,5,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5757291,455,3964,0.5,14,11,474,21,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df.describe()

,Condition,Case,Diameter,Year,Month,Reference,Consumption,industry,housing,agriculture,...,Gauge_Otras_Desconocida,Gauge_Precisa_Ii,Gauge_Recordal,Gauge_Sappel,Gauge_Sappel_Aquarius,Gauge_Sensus,Gauge_Shlumberger,Gauge_Tonhy,Gauge_Yt_Md15,Gauge_Zenner_Half_In
count,1.167268e+07,1.167268e+07,1.167268e+07,1.167268e+07,1.167268e+07,1.167268e+07,1.167268e+07,1.167268e+07,1.167268e+07,1.167268e+07,...,1.167268e+07,1.167268e+07,1.167268e+07,1.167268e+07,1.167268e+07,1.167268e+07,1.167268e+07,1.167268e+07,1.167268e+07,1.167268e+07
mean,3.355588e+02,2.809605e+03,5.031340e-01,1.200000e+01,6.499998e+00,1.391269e+03,1.757733e+01,1.122546e-01,9.972798e-01,1.468044e-03,...,2.155290e-02,1.151407e-04,1.223370e-04,8.563590e-04,2.878518e-05,5.598717e-03,3.218902e-02,5.757035e-05,7.196294e-06,1.223370e-04
std,1.993419e+02,3.630568e+03,5.808043e-02,1.999995e+00,3.452052e+00,6.076382e+03,1.626746e+02,3.156794e-01,5.208455e-02,3.828693e-02,...,1.452184e-01,1.072975e-02,1.105993e-02,2.925108e-02,5.365105e-03,7.461482e-02,1.765018e-01,7.587295e-03,2.682581e-03,1.105993e-02
min,1.000000e+00,1.000000e+00,5.000000e-01,9.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.630000e+02,3.010000e+02,5.000000e-01,1.000000e+01,4.000000e+00,1.290000e+02,9.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.810000e+02,1.189000e+03,5.000000e-01,1.200000e+01,6.000000e+00,5.590000e+02,1.600000e+01,0.000000e+00,1.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,4.550000e+02,4.087000e+03,5.000000e-01,1.400000e+01,9.000000e+00,1.498000e+03,2.200000e+01,0.000000e+00,1.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,9.310000e+02,1.717000e+04,1.000000e+01,1.500000e+01,1.200000e+01,6.940500e+05,1.003020e+05,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


**The Main questions**
- Can we predict the monthly water consumption through the given features?
- Can we predict the reference consumtion through the given features?
- Is the monthly water consumption affected by the time (month-year)?
- What are the most affecting features on the monthly water consumption?
- What are the most affecting features on the reference consumption?

In [8]:
# Dropping unnecessary columns
df.drop(['Condition', 'Case'], axis=1, inplace=True)
df.head(2)

,Diameter,Year,Month,Reference,Consumption,industry,housing,agriculture,commerce,reserve,...,Gauge_Otras_Desconocida,Gauge_Precisa_Ii,Gauge_Recordal,Gauge_Sappel,Gauge_Sappel_Aquarius,Gauge_Sensus,Gauge_Shlumberger,Gauge_Tonhy,Gauge_Yt_Md15,Gauge_Zenner_Half_In
0,0.5,9,1,3761,17,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0.5,9,2,3761,20,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Separate the independent features
data_X = df.drop(['Reference', 'Consumption'], axis=1) 

# The first dependent feature
data_yR = df['Reference']
# The second dependent feature
data_yC = df['Consumption']

In [10]:
# specify the Trainning and testing records for 'Reference' feature
X_trainR, X_testR, y_trainR, y_testR = train_test_split(data_X, 
                                                        data_yR, 
                                                        test_size=0.33, 
                                                        random_state=79)

In [11]:
# specify the Trainning and testing records for 'Consumption' feature
X_trainC, X_testC, y_trainC, y_testC = train_test_split(data_X, 
                                                        data_yC, 
                                                        test_size=0.33, 
                                                        random_state=79)

In [12]:
# define the models
modelR = linear_model.LinearRegression()
modelC = linear_model.LinearRegression()

In [13]:
# Fitting the models
modelR.fit(X_trainR, y_trainR)
modelC.fit(X_trainC, y_trainC)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [14]:
# Make predictions using the testing datasets
y_predR = modelR.predict(X_testR)
y_predC = modelC.predict(X_testC)

In [15]:
# Make predictions using the TRAINING datasets
y_predR_tr = modelR.predict(X_trainR)
y_predC_tr = modelC.predict(X_trainC)

In [16]:
# The coefficients

# ... of 'Reference' feature
intrcptR = modelR.intercept_
coefR = modelR.coef_
# for the testing dataset
mseR = mean_squared_error(y_testR, y_predR)
r2R = r2_score(y_testR, y_predR)
# for the training dataset
mseR_tr = mean_squared_error(y_trainR, y_predR_tr)
r2R_tr = r2_score(y_trainR, y_predR_tr)

# ... of 'Consumption' feature
intrcptC = modelC.intercept_
coefC = modelC.coef_
# for the testing dataset
mseC = mean_squared_error(y_testC, y_predC)
r2C = r2_score(y_testC, y_predC)
# for the training dataset
mseC_tr = mean_squared_error(y_trainC, y_predC_tr)
r2C_tr = r2_score(y_trainC, y_predC_tr)

In [17]:

print(f'''
The metrics for training and testing datasets are:
Feature        Dataset      The MSE            the r2
==========================================================
Reference      Training     {mseR_tr:11.2f}       {r2R_tr*100:05.2f}%
Reference      Testing      {mseR:11.2f}       {r2R*100:05.2f}%
Consumption    Training     {mseC_tr:11.2f}       {r2C_tr*100:05.2f}%
Consumption    Testing      {mseC:11.2f}       {r2C*100:05.2f}%
==========================================================

For the **Reference** feature
The fitting coeefficients are:
    1- The intercept = {intrcptR:6.3f}
    2- The slopes    = 
{coefR}
and For the **Consumption** feature
The fitting coeefficients are:
    1- The intercept = {intrcptC:6.3f}
    2- The slopes    = 
{coefC}
''')


The metrics for training and testing datasets are:
Feature        Dataset      The MSE            the r2
Reference      Training     34721242.46       06.01%
Reference      Testing      34736285.84       05.83%
Consumption    Training         6672.03       75.83%
Consumption    Testing          6867.61       71.56%

For the **Reference** feature
The fitting coeefficients are:
    1- The intercept = -64874898147096.641
    2- The slopes    = 
[ 1.06584295e+04  5.78937534e-01  5.68323124e-02  7.14158468e+02
  3.97075552e+03  5.52286459e+03  1.04651857e+03  9.86140430e+02
  3.25084202e+03  1.61349906e+03  1.78427096e+03  1.32940154e+03
  4.01888821e+13  4.31375301e+13  4.01888821e+13  4.31375301e+13
 -2.94864803e+12 -2.94864803e+12 -2.94864803e+12 -2.94864803e+12
  2.46860160e+13  2.46860160e+13  2.46860160e+13  2.46860160e+13
  2.46860160e+13  2.46860160e+13  2.46860160e+13  2.46860160e+13
  2.46860160e+13  2.46860160e+13  2.46860160e+13  2.46860160e+13
  2.46860160e+13  2.46860160e+13 

In [18]:
# Saving th output parameters of the models

# Create a saving container

models_outputs = {}

In [19]:
# Saving th output parameters of the FIRST model

output = (intrcptR, coefR, mseR_tr, r2R_tr, mseR, r2R, 
            intrcptC, coefC, mseC_tr, r2C_tr, mseC, r2C)
models_outputs['Linear with all variables non normalized '] = output

I found that the coefficients are equal for the last 53 features, which are the gauges types, hence, all the gauges have the sme effect, or they may have small or no effect on the prediction of either feature.

I will repeat the analysis by:
1. Removing the gauges
2. Normalizing the `Diameter`, `Year`, `Month` features 
3. Adding a normalized Date feature by combining the `Year` & `Month`

In [44]:
def perform_fitting_one_model(data_X, data_y, regressor, scale_features=False):
    '''
    A function to make future fittings easier
    inputs:
    * data_X, data_y: Arrays for X, Y
    * regressor: The regression model, for example:
                 linear_model.LinearRegression(normalize=True)

    '''
    # To measure the time
    start_time = time.time()

    # specify the Trainning and testing records for 'Reference' feature
    X_train, X_test, y_train, y_test = train_test_split(data_X, 
                                                            data_y, 
                                                            test_size=0.33, 
                                                            random_state=79)
    if scale_features:
        # Apply scaler
        scaler = StandardScaler()

        # fitting on Training data only
        scaler.fit(X_train)

        # transforming
        X_train, X_test = scaler.transform(X_train), scaler.transform(X_test)

    # define the models
    model = regressor # linear_model.LinearRegression()

    # Fitting the models
    model.fit(X_train, y_train)

    # Make predictions using the TRAINING datasets
    y_pred_tr = model.predict(X_train)
    
    # Make predictions using the testing datasets
    y_pred = model.predict(X_test)
    # The coefficients

    # ... of 'Reference' feature
    intrcpt = model.intercept_
    coef = model.coef_
    # for the testing dataset
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    # for the training dataset
    mse_tr = mean_squared_error(y_train, y_pred_tr)
    r2_tr = r2_score(y_train, y_pred_tr)

    elapsed_time = time.time() - start_time
    print(elapsed_time)
    print(f'The elapsed time for the test is {str(timedelta(seconds=elapsed_time))}')
    
    return (modelR, (intrcpt, coef, mse_tr, r2_tr, mse, r2))

In [31]:
def perform_fitting_two_variables(data_X, data_yR, data_yC, regressor):
    '''
    A function to make future fittings easier
    inputs:
    * data_X, data_yR, data_yC: Arrays for X, Y1, Y2
    * regressor: The regression model, for example:
                 linear_model.LinearRegression(normalize=True)

    '''
    # To measure the time
    start_time = time.time()

    # specify the Trainning and testing records for 'Reference' feature
    X_trainR, X_testR, y_trainR, y_testR = train_test_split(data_X, 
                                                            data_yR, 
                                                            test_size=0.33, 
                                                            random_state=79)
    # specify the Trainning and testing records for 'Consumption' feature
    X_trainC, X_testC, y_trainC, y_testC = train_test_split(data_X, 
                                                            data_yC, 
                                                            test_size=0.33, 
                                                            random_state=79)
    # define the models
    modelR = regressor # linear_model.LinearRegression()
    modelC = regressor # linear_model.LinearRegression()

    # Fitting the models
    modelR.fit(X_trainR, y_trainR)
    modelC.fit(X_trainC, y_trainC)

    # Make predictions using the TRAINING datasets
    y_predR_tr = modelR.predict(X_trainR)
    y_predC_tr = modelC.predict(X_trainC)

    # Make predictions using the testing datasets
    y_predR = modelR.predict(X_testR)
    y_predC = modelC.predict(X_testC)
    # The coefficients

    # ... of 'Reference' feature
    intrcptR = modelR.intercept_
    coefR = modelR.coef_
    # for the testing dataset
    mseR = mean_squared_error(y_testR, y_predR)
    r2R = r2_score(y_testR, y_predR)
    # for the training dataset
    mseR_tr = mean_squared_error(y_trainR, y_predR_tr)
    r2R_tr = r2_score(y_trainR, y_predR_tr)

    # ... of 'Consumption' feature
    intrcptC = modelC.intercept_
    coefC = modelC.coef_
    # for the testing dataset
    mseC = mean_squared_error(y_testC, y_predC)
    r2C = r2_score(y_testC, y_predC)
    # for the training dataset
    mseC_tr = mean_squared_error(y_trainC, y_predC_tr)
    r2C_tr = r2_score(y_trainC, y_predC_tr)

    elapsed_time = time.time() - start_time
    print(elapsed_time)
    print(f'The elapsed time for the test is {str(timedelta(seconds=elapsed_time))}')
    
    return ((modelR, modelC), 
            (intrcptR, coefR, mseR_tr, r2R_tr, mseR, r2R, intrcptC, coefC, mseC_tr, r2C_tr, mseC, r2C))

In [28]:
def print_results(intrcptR, coefR, mseR_tr, r2R_tr, mseR, r2R, 
                  intrcptC, coefC, mseC_tr, r2C_tr, mseC, r2C):
    print(f'''
    The metrics for training and testing datasets are:
    Feature        Dataset      The MSE            the r2
    ==========================================================
    Reference      Training     {mseR_tr:11.2f}       {r2R_tr*100:05.2f}%
    Reference      Testing      {mseR:11.2f}       {r2R*100:05.2f}%
    Consumption    Training     {mseC_tr:11.2f}       {r2C_tr*100:05.2f}%
    Consumption    Testing      {mseC:11.2f}       {r2C*100:05.2f}%
    ==========================================================

    For the **Reference** feature
    The fitting coeefficients are:
        1- The intercept = {intrcptR}
        2- The slopes    = 
    {coefR}
    and For the **Consumption** feature
    The fitting coeefficients are:
        1- The intercept = {intrcptC}
        2- The slopes    = 
    {coefC}
    ''')
    

In [37]:
# Second trial
# Refitting using the new set of features by 
# Normalizing all X values and Removing the Gauge Features

# Select X features, with Removing the Gauge Features
data_X = df[[x for x in list(df) if x[:5]!='Gauge']].drop(['Reference', 'Consumption'], axis=1)
# create a Date_delta column, as integer, to allow regression
# as regression could not be fit on DateTime series

# Assigning Date column
data_X.assign(CurrentDate=0,)
data_X['CurrentDate'] = pd.to_datetime(dict(year=data_X.Year+2000, month=data_X.Month, day=15))
# Creating an integer date value
data_X.assign(date_delta=0)
data_X['date_delta'] = (data_X['CurrentDate'] - data_X['CurrentDate'].min())  / np.timedelta64(1,'D')
# Dropping the initial column
data_X.drop(['CurrentDate'], axis=1, inplace=True)

# The first dependent feature
data_yR = df['Reference']
# The second dependent feature
data_yC = df['Consumption']

# Apply the model
output_R = perform_fitting_one_model(data_X, data_yR, 
                         linear_model.LinearRegression(normalize=True))
coefficients = list(output_R[1])
output_C = perform_fitting_one_model(data_X, data_yC, 
                         linear_model.LinearRegression(normalize=True))
coefficients += list(output_C[1])
# Saving th output parameters of the SECOND model
models_outputs['Linear with non-Gauge variables normalized'] = coefficients
# Printing results
print_results(*coefficients)

10.41742467880249
The elapsed time for the test is 0:00:10.417425
10.63883900642395
The elapsed time for the test is 0:00:10.638839

    The metrics for training and testing datasets are:
    Feature        Dataset      The MSE            the r2
    Reference      Training     36191634.27       02.02%
    Reference      Testing      36175800.03       01.93%
    Consumption    Training        20688.02       25.06%
    Consumption    Testing         18840.00       21.97%

    For the **Reference** feature
    The fitting coeefficients are:
        1- The intercept = 28862222325123.113
        2- The slopes    = 
    [ 1.17349426e+04  9.46150480e+02  7.88293510e+01  6.29363615e+02
  5.00882389e+03  8.17339927e+03  2.97325523e+03  1.98265450e+03
  5.50506795e+03  3.54109150e+03  3.61524336e+03  3.01788609e+03
 -2.88622223e+13  3.94432581e+12 -2.88622223e+13  3.94432581e+12
 -3.28065482e+13 -3.28065482e+13 -3.28065482e+13 -3.28065482e+13
 -2.58952550e+00]
    and For the **Consumption** fea

In [48]:
# Trying to fit with another regressor
# Trying the Stochastic Gradient Descent Regressor

# Separate the independent features
data_X = df.drop(['Reference', 'Consumption'], axis=1) 
# The first dependent feature
data_yR = df['Reference']
# The second dependent feature
data_yC = df['Consumption']

# Apply the model
output_R = perform_fitting_one_model(data_X, data_yR, 
                         linear_model.SGDRegressor(max_iter=50, 
                                                   tol=1e-3, 
                                                   verbose=2), 
                                     scale_features=False)
coefficients = list(output_R[1])
output_C = perform_fitting_one_model(data_X, data_yC, 
                         linear_model.SGDRegressor(max_iter=50, 
                                                   tol=1e-3, 
                                                   verbose=2), 
                                     scale_features=False)
coefficients += list(output_C[1])    
# Saving th output parameters of the SECOND model

models_outputs['SGDRegressor over all data unnormalized'] =coefficients
# Printing results
print_results(*coefficients)

-- Epoch 1
Norm: 14550.70, NNZs: 74, Bias: -1448.151007, T: 7820692, Avg. loss: 18262591.493152
Total training time: 3.33 seconds.
-- Epoch 2
Norm: 16554.76, NNZs: 74, Bias: -2186.877487, T: 15641384, Avg. loss: 18071260.844813
Total training time: 6.62 seconds.
-- Epoch 3
Norm: 18442.62, NNZs: 74, Bias: -2604.683239, T: 23462076, Avg. loss: 18021923.300272
Total training time: 9.66 seconds.
-- Epoch 4
Norm: 19379.74, NNZs: 74, Bias: -2840.784034, T: 31282768, Avg. loss: 17992153.489570
Total training time: 12.65 seconds.
-- Epoch 5
Norm: 20092.70, NNZs: 74, Bias: -2901.002269, T: 39103460, Avg. loss: 17973631.177351
Total training time: 15.59 seconds.
-- Epoch 6
Norm: 20826.31, NNZs: 74, Bias: -3032.795823, T: 46924152, Avg. loss: 17958061.683938
Total training time: 18.52 seconds.
-- Epoch 7
Norm: 21926.80, NNZs: 74, Bias: -3102.863077, T: 54744844, Avg. loss: 17946157.631850
Total training time: 21.46 seconds.
-- Epoch 8
Norm: 21962.90, NNZs: 74, Bias: -3370.208388, T: 62565536, Avg

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:1219: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


177.91932010650635
The elapsed time for the test is 0:02:57.919320
-- Epoch 1
Norm: 1587.11, NNZs: 74, Bias: -383.333766, T: 7820692, Avg. loss: 10839.075678
Total training time: 2.98 seconds.
-- Epoch 2
Norm: 1765.74, NNZs: 74, Bias: -449.396532, T: 15641384, Avg. loss: 10337.936148
Total training time: 5.93 seconds.
-- Epoch 3
Norm: 1750.98, NNZs: 74, Bias: -435.029242, T: 23462076, Avg. loss: 10164.925742
Total training time: 8.87 seconds.
-- Epoch 4
Norm: 2051.15, NNZs: 74, Bias: -482.187936, T: 31282768, Avg. loss: 10122.908610
Total training time: 11.97 seconds.
-- Epoch 5
Norm: 2033.29, NNZs: 74, Bias: -476.677075, T: 39103460, Avg. loss: 10045.723780
Total training time: 15.02 seconds.
-- Epoch 6
Norm: 2184.45, NNZs: 74, Bias: -495.243969, T: 46924152, Avg. loss: 10027.009646
Total training time: 18.04 seconds.
-- Epoch 7
Norm: 2358.96, NNZs: 74, Bias: -508.339833, T: 54744844, Avg. loss: 9946.243822
Total training time: 21.03 seconds.
-- Epoch 8
Norm: 2347.63, NNZs: 74, Bias: 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:1219: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


172.4099497795105
The elapsed time for the test is 0:02:52.409950

    The metrics for training and testing datasets are:
    Feature        Dataset      The MSE            the r2
    Reference      Training     35455202.34       04.02%
    Reference      Testing      35445711.47       03.91%
    Consumption    Training        18982.31       31.24%
    Consumption    Testing         17383.11       28.00%

    For the **Reference** feature
    The fitting coeefficients are:
        1- The intercept = [-4020.85235125]
        2- The slopes    = 
    [ 1.15755898e+04 -6.74443862e+00 -1.47230964e+01  5.08430899e+02
  3.36946877e+03  3.85339778e+03 -6.22481878e+02 -1.07628527e+03
  3.90843405e+03 -2.11309707e+02 -2.94028616e+01 -4.83855475e+02
  5.84309093e+03 -1.53202680e+03 -2.87636572e+03 -1.73178769e+03
 -1.35398262e+03 -1.04712966e+03  5.00605887e+02 -1.36330810e+03
  3.54070200e+02  1.35226531e+02 -6.39340410e+03 -1.67006168e+03
 -2.10602870e+02 -6.60214827e+02 -4.90180195e+02 -6.4890

In [50]:
# Trying to fit with another regressor
# Trying the Stochastic Gradient Descent Regressor


# Select X features, with Removing the Gauge Features
data_X = df[[x for x in list(df) if x[:5]!='Gauge']].drop(['Reference', 'Consumption'], axis=1)

# The first dependent feature
data_yR = df['Reference']
# The second dependent feature
data_yC = df['Consumption']

# Apply the model
output_R = perform_fitting_one_model(data_X, data_yR, 
                         linear_model.SGDRegressor(max_iter=50, 
                                                   tol=1e-3, 
                                                   verbose=2), 
                                     scale_features=False)
coefficients = list(output_R[1])
output_C = perform_fitting_one_model(data_X, data_yC, 
                         linear_model.SGDRegressor(max_iter=50, 
                                                   tol=1e-3, 
                                                   verbose=2), 
                                     scale_features=False)
coefficients += list(output_C[1])    
# Saving th output parameters of the SECOND model

models_outputs['SGDRegressor over non-Gauge data unnormalized'] =coefficients
# Printing results
print_results(*coefficients)

-- Epoch 1
Norm: 13693.23, NNZs: 20, Bias: -2134.510292, T: 7820692, Avg. loss: 18616858.269417
Total training time: 1.95 seconds.
-- Epoch 2
Norm: 14565.12, NNZs: 20, Bias: -2948.407333, T: 15641384, Avg. loss: 18435848.133573
Total training time: 3.99 seconds.
-- Epoch 3
Norm: 14147.92, NNZs: 20, Bias: -3322.815629, T: 23462076, Avg. loss: 18390805.516840
Total training time: 6.00 seconds.
-- Epoch 4
Norm: 14866.38, NNZs: 20, Bias: -3631.876183, T: 31282768, Avg. loss: 18364835.348611
Total training time: 8.01 seconds.
-- Epoch 5
Norm: 14464.87, NNZs: 20, Bias: -3738.106784, T: 39103460, Avg. loss: 18346963.110266
Total training time: 10.03 seconds.
-- Epoch 6
Norm: 14876.35, NNZs: 20, Bias: -3794.042452, T: 46924152, Avg. loss: 18334226.687918
Total training time: 12.04 seconds.
-- Epoch 7
Norm: 14901.46, NNZs: 20, Bias: -3968.031668, T: 54744844, Avg. loss: 18324686.783425
Total training time: 14.05 seconds.
-- Epoch 8
Norm: 15384.36, NNZs: 20, Bias: -4103.758369, T: 62565536, Avg.

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:1219: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


109.64909553527832
The elapsed time for the test is 0:01:49.649096
-- Epoch 1
Norm: 1355.68, NNZs: 20, Bias: -399.155643, T: 7820692, Avg. loss: 10974.866409
Total training time: 1.93 seconds.
-- Epoch 2
Norm: 1489.77, NNZs: 20, Bias: -471.709066, T: 15641384, Avg. loss: 10603.170831
Total training time: 3.86 seconds.
-- Epoch 3
Norm: 1304.06, NNZs: 20, Bias: -468.988586, T: 23462076, Avg. loss: 10582.733624
Total training time: 5.78 seconds.
-- Epoch 4
Norm: 1455.22, NNZs: 20, Bias: -500.690873, T: 31282768, Avg. loss: 10577.823253
Total training time: 7.71 seconds.
-- Epoch 5
Norm: 1340.79, NNZs: 20, Bias: -506.643461, T: 39103460, Avg. loss: 10557.148682
Total training time: 9.72 seconds.
-- Epoch 6
Norm: 1371.95, NNZs: 20, Bias: -520.409202, T: 46924152, Avg. loss: 10534.119277
Total training time: 11.69 seconds.
-- Epoch 7
Norm: 1403.72, NNZs: 20, Bias: -536.716668, T: 54744844, Avg. loss: 10557.420813
Total training time: 13.64 seconds.
-- Epoch 8
Norm: 1335.03, NNZs: 20, Bias: -

In [49]:
# Trying to fit with another regressor
# Trying the Passive Aggressive Regressor

# Separate the independent features
data_X = df.drop(['Reference', 'Consumption'], axis=1) 
# The first dependent feature
data_yR = df['Reference']
# The second dependent feature
data_yC = df['Consumption']

# Apply the model
output_R = perform_fitting_one_model(data_X, data_yR, 
                         linear_model.PassiveAggressiveRegressor(max_iter=50, 
                                                   tol=1e-3, 
                                                   verbose=1), 
                                     scale_features=False)
coefficients = list(output_R[1])
output_C = perform_fitting_one_model(data_X, data_yC, 
                         linear_model.PassiveAggressiveRegressor(max_iter=50, 
                                                   tol=1e-3, 
                                                   verbose=1), 
                                     scale_features=False)
coefficients += list(output_C[1])    
# Saving th output parameters of the SECOND model

models_outputs['PassiveAggressiveRegressor over all data unnormalized'] =coefficients
# Printing results
print_results(*coefficients)

-- Epoch 1
Norm: 4707.37, NNZs: 74, Bias: 552.457842, T: 7820692, Avg. loss: 920.205989
Total training time: 3.33 seconds.
-- Epoch 2
Norm: 5188.20, NNZs: 74, Bias: 504.972008, T: 15641384, Avg. loss: 919.051822
Total training time: 6.59 seconds.
-- Epoch 3
Norm: 5656.16, NNZs: 74, Bias: 514.184430, T: 23462076, Avg. loss: 919.048681
Total training time: 9.80 seconds.
-- Epoch 4
Norm: 5991.21, NNZs: 74, Bias: 502.269748, T: 31282768, Avg. loss: 918.902525
Total training time: 13.07 seconds.
-- Epoch 5
Norm: 6315.73, NNZs: 74, Bias: 496.790036, T: 39103460, Avg. loss: 918.834090
Total training time: 16.33 seconds.
-- Epoch 6
Norm: 6584.06, NNZs: 74, Bias: 508.514550, T: 46924152, Avg. loss: 918.873805
Total training time: 19.55 seconds.
-- Epoch 7
Norm: 6856.74, NNZs: 74, Bias: 498.245882, T: 54744844, Avg. loss: 918.792348
Total training time: 22.72 seconds.
-- Epoch 8
Norm: 7061.45, NNZs: 74, Bias: 509.609632, T: 62565536, Avg. loss: 918.748813
Total training time: 25.90 seconds.
-- E

In [51]:
# Trying to fit with another regressor
# Trying the Passive Aggressive Regressor

# Select X features, with Removing the Gauge Features
data_X = df[[x for x in list(df) if x[:5]!='Gauge']].drop(['Reference', 'Consumption'], axis=1)
# The first dependent feature
data_yR = df['Reference']
# The second dependent feature
data_yC = df['Consumption']

# Apply the model
output_R = perform_fitting_one_model(data_X, data_yR, 
                         linear_model.PassiveAggressiveRegressor(max_iter=50, 
                                                   tol=1e-3, 
                                                   verbose=1), 
                                     scale_features=False)
coefficients = list(output_R[1])
output_C = perform_fitting_one_model(data_X, data_yC, 
                         linear_model.PassiveAggressiveRegressor(max_iter=50, 
                                                   tol=1e-3, 
                                                   verbose=1), 
                                     scale_features=False)
coefficients += list(output_C[1])    
# Saving th output parameters of the SECOND model

models_outputs['PassiveAggressiveRegressor over non-Gauge data unnormalized'] =coefficients
# Printing results
print_results(*coefficients)

-- Epoch 1
Norm: 2794.79, NNZs: 20, Bias: -431.452611, T: 7820692, Avg. loss: 1242.637807
Total training time: 1.78 seconds.
-- Epoch 2
Norm: 2899.48, NNZs: 20, Bias: -470.532913, T: 15641384, Avg. loss: 1242.370200
Total training time: 3.58 seconds.
-- Epoch 3
Norm: 2834.46, NNZs: 20, Bias: -465.428939, T: 23462076, Avg. loss: 1242.252449
Total training time: 5.38 seconds.
-- Epoch 4
Norm: 2857.05, NNZs: 20, Bias: -480.846221, T: 31282768, Avg. loss: 1242.330109
Total training time: 7.16 seconds.
-- Epoch 5
Norm: 2763.89, NNZs: 20, Bias: -501.054285, T: 39103460, Avg. loss: 1242.212986
Total training time: 8.95 seconds.
-- Epoch 6
Norm: 2836.81, NNZs: 20, Bias: -529.678021, T: 46924152, Avg. loss: 1242.227933
Total training time: 10.74 seconds.
-- Epoch 7
Norm: 2847.81, NNZs: 20, Bias: -520.955171, T: 54744844, Avg. loss: 1242.411860
Total training time: 12.54 seconds.
-- Epoch 8
Norm: 2838.94, NNZs: 20, Bias: -525.988177, T: 62565536, Avg. loss: 1242.316827
Total training time: 14.37

In [52]:
models_outputs

{'Linear with all variables non normalized ': (-64874898147096.64,
  array([ 1.06584295e+04,  5.78937534e-01,  5.68323124e-02,  7.14158468e+02,
          3.97075552e+03,  5.52286459e+03,  1.04651857e+03,  9.86140430e+02,
          3.25084202e+03,  1.61349906e+03,  1.78427096e+03,  1.32940154e+03,
          4.01888821e+13,  4.31375301e+13,  4.01888821e+13,  4.31375301e+13,
         -2.94864803e+12, -2.94864803e+12, -2.94864803e+12, -2.94864803e+12,
          2.46860160e+13,  2.46860160e+13,  2.46860160e+13,  2.46860160e+13,
          2.46860160e+13,  2.46860160e+13,  2.46860160e+13,  2.46860160e+13,
          2.46860160e+13,  2.46860160e+13,  2.46860160e+13,  2.46860160e+13,
          2.46860160e+13,  2.46860160e+13,  2.46860160e+13,  2.46860160e+13,
          2.46860160e+13,  2.46860160e+13,  2.46860160e+13,  2.46860160e+13,
          2.46860160e+13,  2.46860160e+13,  2.46860160e+13,  2.46860160e+13,
          2.46860160e+13,  2.46860160e+13,  2.46860160e+13,  2.46860160e+13,
         

In [54]:
# Saving the outputs to desk
np.save('models_outputs.npy', models_outputs) 

In [57]:
# Loading the outputs from desk
read_dictionary = np.load('models_outputs.npy')
read_dictionary.item()

{'Linear with all variables non normalized ': (-64874898147096.64,
  array([ 1.06584295e+04,  5.78937534e-01,  5.68323124e-02,  7.14158468e+02,
          3.97075552e+03,  5.52286459e+03,  1.04651857e+03,  9.86140430e+02,
          3.25084202e+03,  1.61349906e+03,  1.78427096e+03,  1.32940154e+03,
          4.01888821e+13,  4.31375301e+13,  4.01888821e+13,  4.31375301e+13,
         -2.94864803e+12, -2.94864803e+12, -2.94864803e+12, -2.94864803e+12,
          2.46860160e+13,  2.46860160e+13,  2.46860160e+13,  2.46860160e+13,
          2.46860160e+13,  2.46860160e+13,  2.46860160e+13,  2.46860160e+13,
          2.46860160e+13,  2.46860160e+13,  2.46860160e+13,  2.46860160e+13,
          2.46860160e+13,  2.46860160e+13,  2.46860160e+13,  2.46860160e+13,
          2.46860160e+13,  2.46860160e+13,  2.46860160e+13,  2.46860160e+13,
          2.46860160e+13,  2.46860160e+13,  2.46860160e+13,  2.46860160e+13,
          2.46860160e+13,  2.46860160e+13,  2.46860160e+13,  2.46860160e+13,
         